In [1]:
from pathlib import Path
import cv2, math
import numpy as np
import matplotlib.pyplot as plt
import os,cvzone
from tqdm.notebook import tqdm

In [2]:
def imshow(img, showAxis = False, size=(20,10)):
    plt.figure(figsize=size)
    if not showAxis: plt.axis('off')
    if len(img.shape) == 3: plt.imshow(img[:,:,::-1])
    else: plt.imshow(img, cmap='gray')

def BBimgshow(imgFile,bound_box,return_img=False,show_img=True):
    img = cv2.imread(imgFile)
    x,y,w,h= bound_box
    cv2.rectangle(img, (x, y), (x + w, y + h), (0,255,0), 4)
    if show_img: imshow(img)
    if return_img: return img

In [3]:
def rank_cnt(cnt):
    if len(cnt)<5: return 0

    x,y,w,h = cv2.boundingRect(cnt)
    area_boundingRect = w*h
    
    center, axes, angle = cv2.fitEllipse(cnt)
    semi_major_axis, semi_minor_axis = axes[0] / 2.0, axes[1] / 2.0
    area_ellipse = np.pi * semi_major_axis * semi_minor_axis
    ellipse_thres = (area_ellipse/area_boundingRect)

    rect = cv2.minAreaRect(cnt)
    box = cv2.boxPoints(rect)
    box = np.intp(box)

    area_box = cv2.contourArea(box)
    box_thres = area_ellipse/area_box if area_box!=0 else 2

    # threshold = ellipse_thres
    ellipse_axis_thres = semi_major_axis/semi_minor_axis
    ellipse_axis_thres = 1/ellipse_axis_thres if ellipse_axis_thres > 1 else ellipse_axis_thres

    if not(ellipse_thres < 1 and ellipse_axis_thres > 0.47 and box_thres<0.8): return 0
    return ellipse_axis_thres*area_boundingRect+cv2.contourArea(cnt)#+area_ellipse*ellipse_axis_thres)/2


In [4]:
def detect_model(imgFile, debug=False):
    '''
    This function aims to find the pixels in both stamp and number and use their color to identify whether stamp is overlap or not.
    Ideas:
        - Step 1: Extract pixels as a mask has the value of red pixel account for 40%.
        - Step 2:
            2.1 Find contours in the mask
            2.2 Sort the contour based on the score from rank_cnt contour and and keep the best
            2.3 Create a new mask based on selected contour with the ellipse shape
        - Step 3
            3.1 Find the contours in the new mask
            3.2 Get the bounding box of exists contour of stamp
        - Step 4
            - Extract pixels as a mask has the value of black or gray pixel
            - Improve revolution of objects' shape in  mask and find contours in the mask
            - Sort the contours based on the score from rank_cnt contour and and keep top 20
            - Create a new mask based on selected contours with the box shape
        - Step 5
            - Get mask of pixels in both stamp and number
            - Get coordinary of pixels in mask
            - Count the number of red pixels
            - Classify class based on the ratial of red pixels and total

    Args:
        imgFile: An image path
        debug: Print the log if true
    '''

    img_origin = cv2.imread(imgFile)
    img = img_origin.copy()
    if debug: 
        print("Step 1: Detect color")
        imshow(img)

    #=====================================================================
    #Step 1: Extract pixels as a mask has the value of red pixel account for 40%.
    total = img.sum(axis=2)
    total[total==0]=1
    red_channel = img[:,:,2]

    red_mask = np.array((red_channel/total)>0.38, dtype=np.uint8)
    if debug: 
        print("Step 2: Detect color")
        imshow(red_mask)
    #=====================================================================
    #Step 2:
    
    #2.1 Find contours in the mask
    contours, _ = cv2.findContours(image=red_mask.copy(), mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_SIMPLE)
   
    #2.2 Sort the contour based on the score from rank_cnt contour and and keep the best
    topN = 1
    sorted_contours = sorted(contours, key=rank_cnt, reverse=True)
    sorted_contours = sorted_contours[:topN]

    # 2.3 Create a new mask based on selected contour with the ellipse shape
    filteredCircle = np.zeros((img.shape[:2]), dtype =np.uint8)
    stamp_mask = np.zeros((img.shape[:2]), dtype =np.uint8)
    ellipse = cv2.fitEllipse(sorted_contours[0])
    cv2.ellipse(filteredCircle,ellipse,(250,100,100),-1)
    cv2.ellipse(stamp_mask,ellipse,(250,100,100),1)
    if debug: 
        print("Step 3: Detect color")
        imshow(stamp_mask)
    
    #=====================================================================
    # Step 3
    
    #3.1 Find the contours in the new mask
    filteredContours, _ = cv2.findContours(image=filteredCircle.copy(), mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_NONE)
    
    #3.2 Get the bounding box of exists contour
    circleContours = []
    for _, contour in enumerate(filteredContours):
        circleContours.append(contour)
    
    bounds = circleContours[0].squeeze()
    max_x, max_y = bounds[:,0].max(), bounds[:,1].max()
    min_x, min_y = bounds[:,0].min(), bounds[:,1].min()
    bounds = circleContours[0].squeeze()
    w = max_x-min_x
    h = max_y-min_y
    x1, y1 = min_x, min_y

    margin = 0
    # print(img.shape)

    stamp_x1, stamp_y1 = max(0,x1-margin), max(0,y1-margin)
    stamp_x2, stamp_y2 = min(x1+w+margin, img.shape[1]), min(y1+h+margin, img.shape[0])

    new_img = img_origin.copy()[stamp_y1:stamp_y2,stamp_x1:stamp_x2]
    stamp_mask_cut = stamp_mask.copy()[stamp_y1:stamp_y2,stamp_x1:stamp_x2]
    if debug: 
        print("Step 4: Detect stamp_mask_cut")
        imshow(cv2.rectangle(img_origin.copy(), 
                             (stamp_x1, stamp_y1), 
                             (stamp_x2, stamp_y2),
                             (250,200,100),
                             2)
                )
        imshow(new_img)
        imshow(stamp_mask_cut)

    #=====================================================================
    # Step 4

    # Define BGR thresholds for black and gray
    black_lower = np.array([0, 0, 0], dtype=np.uint8)
    black_upper = np.array([95, 95, 95], dtype=np.uint8)

    # Create binary masks based on color thresholds
    black_mask = cv2.inRange(new_img, black_lower, black_upper)

    # Improve revolution of objects' shape in  mask
    kernel1 = np.ones(( 4, 1), np.uint8)
    kernel2 = np.ones(( 1, 1), np.uint8)

    closing = cv2.morphologyEx(black_mask, cv2.MORPH_CLOSE, kernel1,iterations=6)
    sharpen_black_mask = cv2.morphologyEx(closing, cv2.MORPH_OPEN, kernel2,iterations=4)

    d = np.zeros(black_mask.shape)
    t = 6
    d[:black_mask.shape[0]-t,:]=closing[t:,:]
    sharpen_black_mask = np.array(d,dtype=np.uint8)

    if debug: 
        print("Step 5: Detect sharpen_black_mask")
        imshow(sharpen_black_mask)

    # Find the contours in the new mask
    mask = sharpen_black_mask.copy()

    contours_num, _ = cv2.findContours(image=mask.copy(), 
                                       mode=cv2.RETR_TREE, 
                                       method=cv2.CHAIN_APPROX_SIMPLE)
    
    # Sort the contour based on the score from rank_cnt contour and and keep top 20
    topN = 20
    sorted_contours = sorted(contours_num, key=cv2.contourArea, reverse=True)
    sorted_contours = sorted_contours[:topN]

    # Create a new mask based on selected contour with the box shape
    num_mask = np.zeros((mask.shape), dtype=np.uint8)
    edged_mask = np.zeros((mask.shape), dtype=np.uint8)
    thres_num = 5

    global_x1,global_y1 = min_x-stamp_x1, min_y-stamp_y1
    global_x2,global_y2 = stamp_x2+min_x-stamp_x1, stamp_y2+min_y-stamp_y1

    for cnt in sorted_contours:

        x,y,w,h = cv2.boundingRect(cnt)
        area_boundingRect = w*h

        if len(cnt)>thres_num: 
            ellipse = cv2.fitEllipse(cnt)
            # threshold = area_boundingRect
            center, axes, angle = ellipse
            semi_major_axis, semi_minor_axis = axes[0] / 2.0, axes[1] / 2.0
            # Calculate the area of the ellipse
            area_ellipse = np.pi * semi_major_axis * semi_minor_axis
            ellipse_thres = (area_ellipse/area_boundingRect)

            rect = cv2.minAreaRect(cnt)
            box = cv2.boxPoints(rect)
            box = np.intp(box)

            max_x, max_y = box[:,0].max(), box[:,1].max()
            min_x, min_y = box[:,0].min(), box[:,1].min()

            global_x1 = min(global_x1, min_x)
            global_y1 = min(global_y1, min_y)
            global_x2 = max(global_x2, max_x)
            global_y2 = max(global_y2, max_y)

            x,y,w,h = min_x,min_y,max_x-min_x,max_y-min_y

            area_box = cv2.contourArea(box)

            box_thres = area_ellipse/area_box if area_box!=0 else 2
            
            ellipse_axis_thres = semi_major_axis/semi_minor_axis
            ellipse_axis_thres = 1/ellipse_axis_thres if ellipse_axis_thres > 1 else ellipse_axis_thres

            if ellipse_thres>0.78 and ellipse_axis_thres>0.5 and box_thres>0.8 and w/h <0.8:
                cv2.rectangle(edged_mask,(x,y),(x+w,y+h),(250,100,100),-1)
                cv2.rectangle(num_mask,(x,y),(x+w,y+h),(250,100,100),1)

    if debug: 
        print("Step 6: Detect num_mask")
        imshow(edged_mask)
    #=====================================================================
    # Step 5
    # Get mask of pixels in both stamp and number
    out = new_img.copy()
    stamp_num_mask1 = mask.copy()&edged_mask&stamp_mask_cut
    
    # Get coordinary of pixels in mask
    pixel_xy = []
    for row in range(stamp_num_mask1.shape[0]):
        for col in range(stamp_num_mask1.shape[1]):
            if stamp_num_mask1[row,col]:
                pixel_xy.append((row,col))
    color_pixel = cv2.bitwise_and(out, out, mask = stamp_num_mask1)
    if debug: 
        print("Step 7: Detect color_pixel")
        imshow(stamp_num_mask1)
        imshow(color_pixel)

    # Count the number of red pixels
    count1 = 0
    for pid in pixel_xy:
        cp = color_pixel[pid]
        if debug: print(f"{cp} - {cp[-1]/cp.sum() if cp.sum()>0 else 0}")
        count1 += int((cp[-1]/cp.sum() if cp.sum()>0 else 0 )>0.45)
    count1 = (count1,len(pixel_xy))

    # Classify class based on the ratial of red pixels and total
    cls = 0
    a = count1[0]
    b = count1[1]
    if debug: print(f"count: {a}|{b} - {a/b if b>0 else 0}")
    if len(pixel_xy)>2:
        cls = 1
        r = a/b if b>0 else 0
        if r > 0.5: cls = 2
        
    if debug: 
        print("Step 8: Detect stamp_mask_cut")
        imshow(cv2.rectangle(img_origin.copy(), 
                             (stamp_x1+global_x1, stamp_y1+global_y1), 
                             (global_x2-global_x1, global_y2-global_y1),
                             (100,200,250),
                             2)
                )
    return cls, (stamp_x1+global_x1, stamp_y1+global_y1, global_x2-2*global_x1-stamp_x1, global_y2-2*global_y1-stamp_y1)

In [6]:
# error = []

In [7]:
def stamp_detect(dst:str, src_dirs:list, className=["nothing","money_overlap","stamp_overlap"]):
    for src in tqdm(src_dirs,desc="src_dirs"):
        type = os.path.split(src)
        save_dir = os.path.join(dst,type[-1])
        os.makedirs(save_dir,exist_ok=True)
        for root, dir, files in os.walk(src):
            for file in tqdm(files,desc="files",leave=True):
                try:
                    imgFile = os.path.join(root,file)
                    cls,bound_box = detect_model(imgFile)
                    img = BBimgshow(imgFile,bound_box, return_img = True, show_img = False)
                    cvzone.putTextRect(img,f"{className[cls]}",
                                        (max(0,bound_box[0]-40), max(20,bound_box[1]-20)),
                                        scale=2,thickness=2)
                    cv2.imwrite(os.path.join(save_dir,file), img)
                except Exception as e:
                    print(os.path.join(root,file))
                    error.append(os.path.join(root,file))
                    raise e

In [8]:
src = [r"D:\DangNguyen\D_Download\Telegram Desktop\Content_of_test\C008"]
dir =r"D:\DangNguyen\D_Download\Telegram Desktop\Content_of_test\Results"
stamp_detect(dir,src)

src_dirs:   0%|          | 0/1 [00:00<?, ?it/s]

files:   0%|          | 0/161 [00:00<?, ?it/s]

files:   0%|          | 0/4 [00:00<?, ?it/s]